In [5]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
import lxml

In [6]:
df_scraped = pd.read_pickle('data/pickles/scrape.pickle')

# Read in the downloaded filenames
wine_pages = glob.glob('data/wine_pages/wine_*', recursive=True)
wine_pages = np.sort(wine_pages)

In [28]:
df_scraped.head()

,name,image_path,price,url,raw_html
0,Hall Napa Valley Cabernet Sauvignon 2013,/product/images/fl_progressive/158457.jpg,54.99,/product/Hall-Napa-Valley-Cabernet-Sauvignon-2...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
1,Rombauer Chardonnay 2017,/product/images/fl_progressive/wpxkrgx7onirby1...,36.99,/product/Rombauer-Chardonnay-2017/500778,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
2,Silver Oak Napa Valley Cabernet Sauvignon 2014,/product/images/fl_progressive/fog0sb2avopbbq0...,119.99,/product/Silver-Oak-Napa-Valley-Cabernet-Sauvi...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
3,Chateau Cos d'Estournel (Futures Pre-Sale) 2016,/product/images/fl_progressive/202293.jpg,219.97,/product/Chateau-Cos-dEstournel-Futures-Pre-Sa...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
4,Silver Oak Alexander Valley Cabernet Sauvignon...,/product/images/fl_progressive/gpgmugpbf3l6is2...,74.99,/product/Silver-Oak-Alexander-Valley-Cabernet-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."


In [ ]:
# Read, decode, and save wine descriptions & names
names = []
descriptions = []
for page in tqdm(wine_pages):
    #print(page)
    with open(page, "rb") as f:
        #wine = BeautifulSoup(f.read(), parse_only=)
        wine = f.read().decode('utf-8')
    try:  
        names.append(wine.split('><title>')[1].split('|')[0])
    except IndexError:
        continue
    try:
        descriptions.append(wine.split('<div class="viewMoreModule_text"><p>')[1].split('<')[0])
    except IndexError:
        descriptions.append(wine.split('<div class="viewMoreModule_text">')[1].split('<')[0])
        
pd.to_pickle((names,descriptions), path='data/descriptions.pickle')

In [52]:
# Extract Name, Description, and Price information from data
# Some of this is redundant, but it works 
counter = 0
list1 = []
list2 = []
list3 = []
for name in tqdm(descriptions[0]):
    price = df_scraped[df_scraped['name'] == name.strip()]['price'].values
    desc = descriptions[1][counter]
    list1.append(name.strip())
    list2.append(price)
    list3.append(desc)
    counter += 1
    
pd.DataFrame({'name'        : list1,
              'price'       : list2,
              'description' : list3})\
    .to_csv('data/outputs/name_price_desc.csv', index=False, sep='|')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 15549/15549 [00:18<00:00, 855.01it/s]
